In [1]:
from io import open
import os
from conllu import parse
from tqdm import tqdm
import pandas as pd
import re

In [67]:
file = "light_dev.CONLLUA"

In [68]:
data = """"""
max_count = 0
with open(file, 'r', encoding='utf-8') as read_file:
    for line in read_file:
        if line != "\n":
            data += line
        elif line == "\n":
            split_func = lambda line, i: line[i].split("|")
            sentences = parse(data,
                              fields=['id', 'form', 'lemma', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps',
                                      'misc', 'identity', 'bridging', 'discourse_deixis', 'reference', 'nom_sem'],
                              field_parsers={"identity": split_func, "nom_sem": split_func})

            sentence = sentences[0]

In [69]:
print(data)

# global.columns = ID FORM LEMMA UPOS XPOS FEATS HEAD DEPREL DEPS MISC IDENTITY BRIDGING DISCOURSE_DEIXIS REFERENCE NOM_SEM
# newdoc id = light_dev/episode_6686
# setting
# sent_id = episode_6686-1
# text = a ceiling a tree a floor a bag a shield a sword an armor a shoes a vest member orc
1     a        _  _  _  _  _  _  _  _  (EntityID=1|MarkableID=markable_1|Min=2|SemType=dn      _  _  _  (MarkableID=markable_1|Entity_Type=concrete|Genericity=generic-no
2     ceiling  _  _  _  _  _  _  _  _  )                                                       _  _  _  )
3     a        _  _  _  _  _  _  _  _  (EntityID=2|MarkableID=markable_2|Min=4|SemType=dn      _  _  _  (MarkableID=markable_2|Entity_Type=concrete|Genericity=generic-no
4     tree     _  _  _  _  _  _  _  _  )                                                       _  _  _  )
5     a        _  _  _  _  _  _  _  _  (EntityID=3|MarkableID=markable_3|Min=6|SemType=dn      _  _  _  (MarkableID=markable_3|Entity_Type=space|Genericity=ge

In [5]:
df = pd.DataFrame(sentence)
df

,id,form,lemma,upos,xpos,feats,head,deprel,deps,misc,identity,bridging,discourse_deixis,reference,nom_sem
0,1,a,_,_,None,None,None,_,None,None,"[(EntityID=1, MarkableID=markable_1, Min=2, Se...",_,_,_,"[(MarkableID=markable_1, Entity_Type=concrete,..."
1,2,ceiling,_,_,None,None,None,_,None,None,[)],_,_,_,[)]
2,3,a,_,_,None,None,None,_,None,None,"[(EntityID=2, MarkableID=markable_2, Min=4, Se...",_,_,_,"[(MarkableID=markable_2, Entity_Type=concrete,..."
3,4,tree,_,_,None,None,None,_,None,None,[)],_,_,_,[)]
4,5,a,_,_,None,None,None,_,None,None,"[(EntityID=3, MarkableID=markable_3, Min=6, Se...",_,_,_,"[(MarkableID=markable_3, Entity_Type=space, Ge..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11486,591,.,_,_,None,None,None,_,None,None,[_],_,_,_,NaN
11487,592,Buy,_,_,None,None,None,_,None,None,[_],_,_,_,NaN
11488,593,them,_,_,None,None,None,_,None,None,"[(EntityID=35, MarkableID=markable_128, Min=57...",_,_,_,"[(MarkableID=markable_128, Entity_Type=person,..."
11489,594,milk,_,_,None,None,None,_,None,None,"[(EntityID=94, MarkableID=markable_212, Min=59...",_,_,_,"[(MarkableID=markable_212, Entity_Type=substan..."


In [6]:
df = df[['form','identity']]
df.index = df.index + 1

In [62]:
entity = {}

open_stack_index = []
stack_token = []

data = """"""
max_count = 0
with open(file, 'r', encoding='utf-8') as read_file:
    for line in read_file:
        if line != "\n":
            data += line
        elif line == "\n":
            split_func = lambda line, i: line[i].split("|")
            sentences = parse(data,
                              fields=['id', 'form', 'lemma', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps',
                                      'misc', 'identity', 'bridging', 'discourse_deixis', 'reference', 'nom_sem'],
                              field_parsers={"identity": split_func, "nom_sem": split_func})

            sentence = sentences[0]
            for index, token in enumerate(sentence):
                identities = token['identity']
                for identity in identities:
                    if "(" in identity:
                        stack_token.append(str(token))
                        open_stack_index.append(index)
                    elif ")" in identity:
                        count = str(identity).count(")")
                        end_index = index
                        for i in range(0, count):
                            if len(open_stack_index) == 0:
                                print("Do not match !")
                            else:
                                start_index = open_stack_index.pop()
                                mention = ""
                                for i in range(start_index, end_index+1):
                                    mention += sentence[i]["form"] + " "
                                mention = mention.strip()
                                if mention not in entity:
                                    entity[mention] = len(entity)
            data = """"""

In [58]:
entities = list(entity.keys())
entities

['a ceiling',
 'a tree',
 'a floor',
 'a bag',
 'a shield',
 'a sword',
 'an armor',
 'a shoes',
 'a vest',
 'member',
 'orc',
 'I',
 'my',
 'my way',
 'this cave',
 'who',
 'me',
 'the tower',
 'so much trouble',
 'there',
 'you',
 'my cave',
 'A special moss',
 'here in this cave',
 'it',
 'the tower that I work in',
 'the cave',
 'The guano',
 'some fantastic moss',
 'here',
 'now',
 'we',
 'the entrance',
 'The light from the entrance',
 'the moss',
 'The darker places of the cave',
 'moss',
 'course',
 'the outside',
 'those from the outside',
 'they',
 'them',
 'They',
 'a vegan',
 'many years ago',
 'Orcs',
 'many',
 'this very cave',
 'most Orcs',
 'many friends',
 'we all',
 'outsiders',
 'so few',
 'Most people',
 'these dark and damp places',
 'the busy tower',
 'a torch',
 'your',
 'your way',
 'my friend',
 'twists',
 'turns',
 'Here',
 'this one',
 'us',
 'your torch',
 'this small tunnel',
 'No one',
 'kind Orc',
 'the far reaches of this cave',
 'far scarier creatures l

In [10]:
entity_df = pd.DataFrame({"entity": entities}, dtype=object)
entity_df

,entity
0,a ceiling
1,a tree
2,a floor
3,a bag
4,a shield
...,...
1569,the sick
1570,the lepers
1571,these golden halls
1572,you words


In [11]:
entity_df['index'] = ['0']*len(entity_df)
entity_df['head'] = ['0']*len(entity_df)
entity_df

,entity,index,head
0,a ceiling,0,0
1,a tree,0,0
2,a floor,0,0
3,a bag,0,0
4,a shield,0,0
...,...,...,...
1569,the sick,0,0
1570,the lepers,0,0
1571,these golden halls,0,0
1572,you words,0,0


In [75]:
token_df = df.copy()
token_df.head(10)

,form,identity
1,a,"[(EntityID=1, MarkableID=markable_1, Min=2, Se..."
2,ceiling,[)]
3,a,"[(EntityID=2, MarkableID=markable_2, Min=4, Se..."
4,tree,[)]
5,a,"[(EntityID=3, MarkableID=markable_3, Min=6, Se..."
6,floor,[)]
7,a,"[(EntityID=4, MarkableID=markable_4, Min=8, Se..."
8,bag,[)]
9,a,"[(EntityID=5, MarkableID=markable_5, Min=10, S..."
10,shield,[)]


In [13]:
# entity id 

# entity = 'a ceiling'
# entity_index = entity_df.index[(entity_df['entity']==entity)].tolist()[0]
# # print(entity_index)

# idx_li = []
# for token in entity.split(" "):
#     index = token_df.index[(token_df['form']==token)].tolist()[0]
#     idx_li.append(index)    

# print(idx_li)
# entity_df.loc[entity_index, 'index'] = idx_li
# token_df = token_df.drop(index=idx_li, axis=0)

In [14]:
# unused_token = token_df.index[token_df['identity'].apply(lambda x:x == ['_'])].tolist()
# token_df[(token_df['identity'].apply(lambda x:x != ['_'])) & (token_df['form']=='a')]

In [15]:
# len(unused_token)

In [16]:
# token_df = token_df.drop(index=unused_token, axis=0)
# token_df

In [48]:
# 중복 entity 구분 X
identity_li = token_df.identity.tolist()

identities = []   
token_index = []

temp_id = []
temp_tk = []

l_count = 0
r_count = 0
for index, identity in enumerate(identity_li):
    l_count += ''.join(identity).count('(')
    r_count += ''.join(identity).count(')')
    
    if l_count == r_count and identity == ['_']:
        continue
        
    temp_id += identity
    temp_tk.append(index+1)

    if l_count == r_count:
        identities.append(''.join(temp_id))
        token_index.append(temp_tk)            

        temp_id = []
        temp_tk = []

In [142]:
# 중복 entity 분리
identity_li = token_df.identity.tolist()

identities = []   
token_index = []

temp_id = []
temp_tk = []

l_count = 0
r_count = 0
for index, identity in enumerate(identity_li):
    print('index, identity:', index+1, identity)
    l_count += ''.join(identity).count('(')
    r_count += ''.join(identity).count(')')
    print('l_count:', l_count)
    print('r_count:', r_count)
    
    if l_count == r_count and identity == ['_']:
        l_count = 0
        r_count = 0
        continue
        
    temp_id += identity
    temp_tk.append(index+1)

    if l_count == r_count:   # 중복 X
        if l_count == 1:
            identities.append(''.join(temp_id))
            token_index.append(temp_tk)      
            print('identities:', identities)
            print('token_index:', token_index)

            temp_id = []
            temp_tk = []
            l_count = 0
            r_count = 0
            
        else:   # 중복
            redundant_id = ''.join(temp_id)
            print('temp index:', temp_id)
            for i in range(l_count):
                temp_redundant_id = redundant_id[redundant_id.rindex('('):redundant_id.index(')')+1]
                redundant_id = redundant_id.replace(temp_redundant_id,'')
                identities.append(temp_redundant_id)
#                 token_index.append(temp)
                
                print('***redundant***')
                print('temp redundant id:', temp_redundant_id)
                print('identities:', identities)
                
    print('='*50)
            

index, identity: 1 ['(EntityID=1', 'MarkableID=markable_1', 'Min=2', 'SemType=dn']
l_count: 1
r_count: 0
index, identity: 2 [')']
l_count: 0
r_count: 1
index, identity: 3 ['(EntityID=2', 'MarkableID=markable_2', 'Min=4', 'SemType=dn']
l_count: 1
r_count: 0
index, identity: 4 [')']
l_count: 0
r_count: 1
index, identity: 5 ['(EntityID=3', 'MarkableID=markable_3', 'Min=6', 'SemType=dn']
l_count: 1
r_count: 0
index, identity: 6 [')']
l_count: 0
r_count: 1
index, identity: 7 ['(EntityID=4', 'MarkableID=markable_4', 'Min=8', 'SemType=dn']
l_count: 1
r_count: 0
index, identity: 8 [')']
l_count: 0
r_count: 1
index, identity: 9 ['(EntityID=5', 'MarkableID=markable_5', 'Min=10', 'SemType=dn']
l_count: 1
r_count: 0
index, identity: 10 [')']
l_count: 0
r_count: 1
index, identity: 11 ['(EntityID=6', 'MarkableID=markable_6', 'Min=12', 'SemType=dn']
l_count: 1
r_count: 0
index, identity: 12 [')']
l_count: 0
r_count: 1
index, identity: 13 ['(EntityID=7', 'MarkableID=markable_7', 'Min=14', 'SemType=dn'

index, identity: 39 ['(EntityID=15', 'MarkableID=markable_26', 'Min=20', 'SemType=dn']
l_count: 1
r_count: 0
index, identity: 40 [')']
l_count: 0
r_count: 1
index, identity: 41 ['_']
l_count: 0
r_count: 0
index, identity: 42 ['(EntityID=10', 'MarkableID=markable_142', 'Min=22', 'SemType=do)']
l_count: 1
r_count: 1
identities: ['(EntityID=1MarkableID=markable_1Min=2SemType=dn)(EntityID=2MarkableID=markable_2Min=4SemType=dn)(EntityID=3MarkableID=markable_3Min=6SemType=dn)(EntityID=4MarkableID=markable_4Min=8SemType=dn)(EntityID=5MarkableID=markable_5Min=10SemType=dn)(EntityID=6MarkableID=markable_6Min=12SemType=dn)(EntityID=7MarkableID=markable_7Min=14SemType=dn)(EntityID=8MarkableID=markable_8Min=16SemType=dn)(EntityID=9MarkableID=markable_9Min=18SemType=dn)(EntityID=10MarkableID=markable_10Min=19SemType=dn)', '(EntityID=11MarkableID=markable_11Min=20SemType=dn)', '(EntityID=10MarkableID=markable_156Min=2SemType=do)', '(EntityID=12MarkableID=markable_108Min=6SemType=dn(EntityID=10Markab

identities: ['(EntityID=1MarkableID=markable_1Min=2SemType=dn)(EntityID=2MarkableID=markable_2Min=4SemType=dn)(EntityID=3MarkableID=markable_3Min=6SemType=dn)(EntityID=4MarkableID=markable_4Min=8SemType=dn)(EntityID=5MarkableID=markable_5Min=10SemType=dn)(EntityID=6MarkableID=markable_6Min=12SemType=dn)(EntityID=7MarkableID=markable_7Min=14SemType=dn)(EntityID=8MarkableID=markable_8Min=16SemType=dn)(EntityID=9MarkableID=markable_9Min=18SemType=dn)(EntityID=10MarkableID=markable_10Min=19SemType=dn)', '(EntityID=11MarkableID=markable_11Min=20SemType=dn)', '(EntityID=10MarkableID=markable_156Min=2SemType=do)', '(EntityID=12MarkableID=markable_108Min=6SemType=dn(EntityID=10MarkableID=markable_44Min=6SemType=do))(EntityID=13MarkableID=markable_32Min=10SemType=dn)(EntityID=14-PseudoMarkableID=markable_107Min=13SemType=quantifier)', '(EntityID=10MarkableID=markable_128Min=16SemType=do)', '(EntityID=15MarkableID=markable_26Min=20SemType=dn)(EntityID=10MarkableID=markable_142Min=22SemType=do)',

identities: ['(EntityID=1MarkableID=markable_1Min=2SemType=dn)(EntityID=2MarkableID=markable_2Min=4SemType=dn)(EntityID=3MarkableID=markable_3Min=6SemType=dn)(EntityID=4MarkableID=markable_4Min=8SemType=dn)(EntityID=5MarkableID=markable_5Min=10SemType=dn)(EntityID=6MarkableID=markable_6Min=12SemType=dn)(EntityID=7MarkableID=markable_7Min=14SemType=dn)(EntityID=8MarkableID=markable_8Min=16SemType=dn)(EntityID=9MarkableID=markable_9Min=18SemType=dn)(EntityID=10MarkableID=markable_10Min=19SemType=dn)', '(EntityID=11MarkableID=markable_11Min=20SemType=dn)', '(EntityID=10MarkableID=markable_156Min=2SemType=do)', '(EntityID=12MarkableID=markable_108Min=6SemType=dn(EntityID=10MarkableID=markable_44Min=6SemType=do))(EntityID=13MarkableID=markable_32Min=10SemType=dn)(EntityID=14-PseudoMarkableID=markable_107Min=13SemType=quantifier)', '(EntityID=10MarkableID=markable_128Min=16SemType=do)', '(EntityID=15MarkableID=markable_26Min=20SemType=dn)(EntityID=10MarkableID=markable_142Min=22SemType=do)',

identities: ['(EntityID=1MarkableID=markable_1Min=2SemType=dn)(EntityID=2MarkableID=markable_2Min=4SemType=dn)(EntityID=3MarkableID=markable_3Min=6SemType=dn)(EntityID=4MarkableID=markable_4Min=8SemType=dn)(EntityID=5MarkableID=markable_5Min=10SemType=dn)(EntityID=6MarkableID=markable_6Min=12SemType=dn)(EntityID=7MarkableID=markable_7Min=14SemType=dn)(EntityID=8MarkableID=markable_8Min=16SemType=dn)(EntityID=9MarkableID=markable_9Min=18SemType=dn)(EntityID=10MarkableID=markable_10Min=19SemType=dn)', '(EntityID=11MarkableID=markable_11Min=20SemType=dn)', '(EntityID=10MarkableID=markable_156Min=2SemType=do)', '(EntityID=12MarkableID=markable_108Min=6SemType=dn(EntityID=10MarkableID=markable_44Min=6SemType=do))(EntityID=13MarkableID=markable_32Min=10SemType=dn)(EntityID=14-PseudoMarkableID=markable_107Min=13SemType=quantifier)', '(EntityID=10MarkableID=markable_128Min=16SemType=do)', '(EntityID=15MarkableID=markable_26Min=20SemType=dn)(EntityID=10MarkableID=markable_142Min=22SemType=do)',

...

identities: ['(EntityID=1MarkableID=markable_1Min=2SemType=dn)(EntityID=2MarkableID=markable_2Min=4SemType=dn)(EntityID=3MarkableID=markable_3Min=6SemType=dn)(EntityID=4MarkableID=markable_4Min=8SemType=dn)(EntityID=5MarkableID=markable_5Min=10SemType=dn)(EntityID=6MarkableID=markable_6Min=12SemType=dn)(EntityID=7MarkableID=markable_7Min=14SemType=dn)(EntityID=8MarkableID=markable_8Min=16SemType=dn)(EntityID=9MarkableID=markable_9Min=18SemType=dn)(EntityID=10MarkableID=markable_10Min=19SemType=dn)', '(EntityID=11MarkableID=markable_11Min=20SemType=dn)', '(EntityID=10MarkableID=markable_156Min=2SemType=do)', '(EntityID=12MarkableID=markable_108Min=6SemType=dn(EntityID=10MarkableID=markable_44Min=6SemType=do))(EntityID=13MarkableID=markable_32Min=10SemType=dn)(EntityID=14-PseudoMarkableID=markable_107Min=13SemType=quantifier)', '(EntityID=10MarkableID=markable_128Min=16SemType=do)', '(EntityID=15MarkableID=markable_26Min=20SemType=dn)(EntityID=10MarkableID=markable_142Min=22SemType=do)',

identities: ['(EntityID=1MarkableID=markable_1Min=2SemType=dn)(EntityID=2MarkableID=markable_2Min=4SemType=dn)(EntityID=3MarkableID=markable_3Min=6SemType=dn)(EntityID=4MarkableID=markable_4Min=8SemType=dn)(EntityID=5MarkableID=markable_5Min=10SemType=dn)(EntityID=6MarkableID=markable_6Min=12SemType=dn)(EntityID=7MarkableID=markable_7Min=14SemType=dn)(EntityID=8MarkableID=markable_8Min=16SemType=dn)(EntityID=9MarkableID=markable_9Min=18SemType=dn)(EntityID=10MarkableID=markable_10Min=19SemType=dn)', '(EntityID=11MarkableID=markable_11Min=20SemType=dn)', '(EntityID=10MarkableID=markable_156Min=2SemType=do)', '(EntityID=12MarkableID=markable_108Min=6SemType=dn(EntityID=10MarkableID=markable_44Min=6SemType=do))(EntityID=13MarkableID=markable_32Min=10SemType=dn)(EntityID=14-PseudoMarkableID=markable_107Min=13SemType=quantifier)', '(EntityID=10MarkableID=markable_128Min=16SemType=do)', '(EntityID=15MarkableID=markable_26Min=20SemType=dn)(EntityID=10MarkableID=markable_142Min=22SemType=do)',

identities: ['(EntityID=1MarkableID=markable_1Min=2SemType=dn)(EntityID=2MarkableID=markable_2Min=4SemType=dn)(EntityID=3MarkableID=markable_3Min=6SemType=dn)(EntityID=4MarkableID=markable_4Min=8SemType=dn)(EntityID=5MarkableID=markable_5Min=10SemType=dn)(EntityID=6MarkableID=markable_6Min=12SemType=dn)(EntityID=7MarkableID=markable_7Min=14SemType=dn)(EntityID=8MarkableID=markable_8Min=16SemType=dn)(EntityID=9MarkableID=markable_9Min=18SemType=dn)(EntityID=10MarkableID=markable_10Min=19SemType=dn)', '(EntityID=11MarkableID=markable_11Min=20SemType=dn)', '(EntityID=10MarkableID=markable_156Min=2SemType=do)', '(EntityID=12MarkableID=markable_108Min=6SemType=dn(EntityID=10MarkableID=markable_44Min=6SemType=do))(EntityID=13MarkableID=markable_32Min=10SemType=dn)(EntityID=14-PseudoMarkableID=markable_107Min=13SemType=quantifier)', '(EntityID=10MarkableID=markable_128Min=16SemType=do)', '(EntityID=15MarkableID=markable_26Min=20SemType=dn)(EntityID=10MarkableID=markable_142Min=22SemType=do)',

identities: ['(EntityID=1MarkableID=markable_1Min=2SemType=dn)(EntityID=2MarkableID=markable_2Min=4SemType=dn)(EntityID=3MarkableID=markable_3Min=6SemType=dn)(EntityID=4MarkableID=markable_4Min=8SemType=dn)(EntityID=5MarkableID=markable_5Min=10SemType=dn)(EntityID=6MarkableID=markable_6Min=12SemType=dn)(EntityID=7MarkableID=markable_7Min=14SemType=dn)(EntityID=8MarkableID=markable_8Min=16SemType=dn)(EntityID=9MarkableID=markable_9Min=18SemType=dn)(EntityID=10MarkableID=markable_10Min=19SemType=dn)', '(EntityID=11MarkableID=markable_11Min=20SemType=dn)', '(EntityID=10MarkableID=markable_156Min=2SemType=do)', '(EntityID=12MarkableID=markable_108Min=6SemType=dn(EntityID=10MarkableID=markable_44Min=6SemType=do))(EntityID=13MarkableID=markable_32Min=10SemType=dn)(EntityID=14-PseudoMarkableID=markable_107Min=13SemType=quantifier)', '(EntityID=10MarkableID=markable_128Min=16SemType=do)', '(EntityID=15MarkableID=markable_26Min=20SemType=dn)(EntityID=10MarkableID=markable_142Min=22SemType=do)',

KeyboardInterrupt: 

In [133]:
redundant_id = '(EntityID=13MarkableID=markable_47Min=50SemType=do(EntityID=11MarkableID=markable_136Min=49SemType=doElementOf=23))'
# print(redundant_id.rindex('('))
l_count = 2
identities = []
for i in range(l_count):
    temp_redundant_id = redundant_id[redundant_id.rindex('('):redundant_id.index(')')+1]
    redundant_id = redundant_id.replace(temp_redundant_id,'')
#     print(redundant_id)
#     redundant_id = redundant_id[:redundant_id.rindex('(')]

(EntityID=11MarkableID=markable_136Min=49SemType=doElementOf=23)
(EntityID=13MarkableID=markable_47Min=50SemType=do)


In [49]:
len(identities)
len(token_index)
# token_index

2894

In [46]:
token_index

[[1, 2],
 [3, 4],
 [5, 6],
 [7, 8],
 [9, 10],
 [11, 12],
 [13, 14],
 [15, 16],
 [17, 18],
 [19],
 [20],
 [22],
 [26, 27],
 [30, 31],
 [33],
 [36],
 [39, 40],
 [42],
 [48, 49, 50],
 [53],
 [55],
 [58],
 [63],
 [69, 70],
 [72, 73, 74],
 [76, 77, 78, 79],
 [81],
 [83],
 [86, 87, 88, 89, 90, 91],
 [93],
 [99, 100],
 [103],
 [105],
 [108],
 [115, 116],
 [124, 125],
 [128, 129, 130],
 [133],
 [136],
 [138, 139, 140],
 [144],
 [148, 149, 150, 151],
 [153],
 [159, 160],
 [162, 163, 164, 165, 166],
 [168, 169],
 [170],
 [176, 177, 178, 179, 180, 181],
 [184, 185, 186, 187],
 [189],
 [194],
 [197],
 [199],
 [202],
 [204, 205, 206],
 [209],
 [213],
 [216, 217, 218, 219],
 [224],
 [226],
 [230],
 [233],
 [235],
 [240],
 [242, 243],
 [244, 245, 246],
 [248],
 [252],
 [256],
 [258],
 [261],
 [263, 264, 265],
 [267],
 [270, 271],
 [277],
 [280],
 [283, 284],
 [286],
 [289, 290],
 [292, 293, 294, 295],
 [298, 299],
 [303],
 [311, 312],
 [317, 318, 319, 320, 321],
 [323],
 [325],
 [335, 336, 337],
 [34

In [50]:
new_df = pd.DataFrame(columns=['entity', 'index', 'identity', 'head'])
new_df

,entity,index,identity,head


In [51]:
test_df = new_df.copy()
test_df

,entity,index,identity,head


In [52]:
new_df['index'] = token_index
new_df['identity'] = identities
new_df.index = new_df.index+1
new_df

,entity,index,identity,head
1,NaN,"[1, 2]",(EntityID=1MarkableID=markable_1Min=2SemType=dn),NaN
2,NaN,"[3, 4]",(EntityID=2MarkableID=markable_2Min=4SemType=dn),NaN
3,NaN,"[5, 6]",(EntityID=3MarkableID=markable_3Min=6SemType=dn),NaN
4,NaN,"[7, 8]",(EntityID=4MarkableID=markable_4Min=8SemType=dn),NaN
5,NaN,"[9, 10]",(EntityID=5MarkableID=markable_5Min=10SemType=dn),NaN
...,...,...,...,...
2890,NaN,[11480],(EntityID=93MarkableID=markable_134Min=569SemT...,NaN
2891,NaN,[11483],(EntityID=83MarkableID=markable_166Min=572SemT...,NaN
2892,NaN,"[11485, 11486]",(EntityID=35MarkableID=markable_74Min=574SemTy...,NaN
2893,NaN,[11489],(EntityID=35MarkableID=markable_128Min=578SemT...,NaN


In [53]:
new_df['entity'] = ['0']*len(new_df)

In [54]:
new_df.head(50)

,entity,index,identity,head
1,0,"[1, 2]",(EntityID=1MarkableID=markable_1Min=2SemType=dn),NaN
2,0,"[3, 4]",(EntityID=2MarkableID=markable_2Min=4SemType=dn),NaN
3,0,"[5, 6]",(EntityID=3MarkableID=markable_3Min=6SemType=dn),NaN
4,0,"[7, 8]",(EntityID=4MarkableID=markable_4Min=8SemType=dn),NaN
5,0,"[9, 10]",(EntityID=5MarkableID=markable_5Min=10SemType=dn),NaN
6,0,"[11, 12]",(EntityID=6MarkableID=markable_6Min=12SemType=dn),NaN
7,0,"[13, 14]",(EntityID=7MarkableID=markable_7Min=14SemType=dn),NaN
8,0,"[15, 16]",(EntityID=8MarkableID=markable_8Min=16SemType=dn),NaN
9,0,"[17, 18]",(EntityID=9MarkableID=markable_9Min=18SemType=dn),NaN
10,0,[19],(EntityID=10MarkableID=markable_10Min=19SemTyp...,NaN


In [70]:
sentences

[TokenList<a, ceiling, a, tree, a, floor, a, bag, a, shield, a, sword, an, armor, a, shoes, a, vest, member, orc, If, I, can, not, find, my, way, out, of, this, cave, ,, who, will, replace, me, working, in, the, tower, ?, I, am, going, to, get, into, so, much, trouble, ., Hello, there, !, I, can, help, you, out, !, How, did, you, end, up, so, far, into, my, cave, ?, A, special, moss, grows, here, in, this, cave, ., I, use, it, to, clean, the, tower, that, I, work, in, ., I, need, to, travel, further, into, the, cave, to, find, it, but, I, m, afraid, I, wo, nt, be, able, to, find, my, way, out, again, !, Ahhhhh, ,, yes, !, The, guano, does, encourage, some, fantastic, moss, to, grow, here, !, Do, you, have, enough, for, now, ?, Or, do, we, need, for, find, some, more, for, you, ?, I, need, to, travel, further, into, the, cave, ., The, light, from, the, entrance, causes, the, moss, here, to, grow, very, small, ., The, darker, places, of, the, cave, will, have, wonderful, patches, of, mos

In [71]:
sentences[0][0]['form']

'a'

In [72]:
index_li = new_df['index'].tolist()

for index, entity_idx in enumerate(index_li):
    
    temp_token = []
    for token_idx in entity_idx:
        temp_token.append(sentences[0][token_idx-1]['form'])
        
    new_df.loc[index+1, 'entity'] = ' '.join(temp_token)

In [73]:
new_df

,entity,index,identity,head
1,a ceiling,"[1, 2]",(EntityID=1MarkableID=markable_1Min=2SemType=dn),NaN
2,a tree,"[3, 4]",(EntityID=2MarkableID=markable_2Min=4SemType=dn),NaN
3,a floor,"[5, 6]",(EntityID=3MarkableID=markable_3Min=6SemType=dn),NaN
4,a bag,"[7, 8]",(EntityID=4MarkableID=markable_4Min=8SemType=dn),NaN
5,a shield,"[9, 10]",(EntityID=5MarkableID=markable_5Min=10SemType=dn),NaN
...,...,...,...,...
2890,them,[11480],(EntityID=93MarkableID=markable_134Min=569SemT...,NaN
2891,bread,[11483],(EntityID=83MarkableID=markable_166Min=572SemT...,NaN
2892,the poor,"[11485, 11486]",(EntityID=35MarkableID=markable_74Min=574SemTy...,NaN
2893,them,[11489],(EntityID=35MarkableID=markable_128Min=578SemT...,NaN


In [74]:
new_df.head(50)

,entity,index,identity,head
1,a ceiling,"[1, 2]",(EntityID=1MarkableID=markable_1Min=2SemType=dn),NaN
2,a tree,"[3, 4]",(EntityID=2MarkableID=markable_2Min=4SemType=dn),NaN
3,a floor,"[5, 6]",(EntityID=3MarkableID=markable_3Min=6SemType=dn),NaN
4,a bag,"[7, 8]",(EntityID=4MarkableID=markable_4Min=8SemType=dn),NaN
5,a shield,"[9, 10]",(EntityID=5MarkableID=markable_5Min=10SemType=dn),NaN
6,a sword,"[11, 12]",(EntityID=6MarkableID=markable_6Min=12SemType=dn),NaN
7,an armor,"[13, 14]",(EntityID=7MarkableID=markable_7Min=14SemType=dn),NaN
8,a shoes,"[15, 16]",(EntityID=8MarkableID=markable_8Min=16SemType=dn),NaN
9,a vest,"[17, 18]",(EntityID=9MarkableID=markable_9Min=18SemType=dn),NaN
10,member,[19],(EntityID=10MarkableID=markable_10Min=19SemTyp...,NaN


In [91]:
entity_df.entity

0                a ceiling
1                   a tree
2                  a floor
3                    a bag
4                 a shield
               ...        
1569              the sick
1570            the lepers
1571    these golden halls
1572             you words
1573                  milk
Name: entity, Length: 1574, dtype: object

In [94]:
new_df.entity

1       a ceiling
2          a tree
3         a floor
4           a bag
5        a shield
          ...    
2890         them
2891        bread
2892     the poor
2893         them
2894         milk
Name: entity, Length: 2894, dtype: object

In [413]:
# 중복 entity
for index, identity in enumerate(identities):
    if identity.count('(') > 1 and identity.count(')') > 1:
        print(identity)
        print(new_df.loc[index, 'entity'])

(EntityID=12MarkableID=markable_108Min=6SemType=dn(EntityID=10MarkableID=markable_44Min=6SemType=do))
I
(EntityID=13MarkableID=markable_47Min=50SemType=do(EntityID=11MarkableID=markable_136Min=49SemType=doElementOf=23))
you
(EntityID=19MarkableID=markable_162Min=76,79SemType=dn_(EntityID=13MarkableID=markable_90Min=59SemType=do))
A special moss
(EntityID=15MarkableID=markable_19Min=70SemType=do__(EntityID=10MarkableID=markable_36Min=69SemType=do)_)
it
(EntityID=12MarkableID=markable_124Min=96SemType=do(EntityID=10MarkableID=markable_20Min=95SemType=do))
I
(EntityID=21-PseudoMarkableID=markable_38Min=120SemType=quantifier_)(EntityID=22MarkableID=markable_151Min=120SemType=dn)
you
(EntityID=18MarkableID=markable_146Min=130SemType=do__)(EntityID=10MarkableID=markable_98Min=131SemType=do)
we
(EntityID=24MarkableID=markable_14Min=146SemType=dn__(EntityID=25MarkableID=markable_45Min=146SemType=dn))
the cave
(EntityID=27MarkableID=markable_51Min=160SemType=dn___(EntityID=13MarkableID=markable

(EntityID=64MarkableID=markable_41Min=447SemType=do(EntityID=70MarkableID=markable_163Min=446SemType=do))
proof
(EntityID=80MarkableID=markable_237Min=508,514SemType=dn(EntityID=64MarkableID=markable_139Min=449SemType=do)__(EntityID=81MarkableID=markable_176Min=453SemType=dn_(EntityID=70MarkableID=markable_196Min=454SemType=do)))
our family
(EntityID=82MarkableID=markable_223Min=464SemType=dn__(EntityID=11MarkableID=markable_52Min=464SemType=do_))
its mark on every building we built
(EntityID=83MarkableID=markable_59Min=472SemType=dn__(EntityID=84MarkableID=markable_154Min=472SemType=dn__(EntityID=9MarkableID=markable_28Min=472SemType=do)))
the mural in the public bath
(EntityID=80MarkableID=markable_58Min=476SemType=do(EntityID=70MarkableID=markable_119Min=475SemType=do))
The inscriptions on the side of the fountain
(EntityID=85MarkableID=markable_238Min=548,550SemType=dn(EntityID=24MarkableID=markable_86Min=490SemType=do))
a holy man
(EntityID=80MarkableID=markable_168Min=498SemType=

(EntityID=83-PseudoMarkableID=markable_193Min=395,402SemType=quantifier_(EntityID=10MarkableID=markable_40Min=374SemType=do)__(EntityID=84MarkableID=markable_98Min=379SemType=dn(EntityID=10MarkableID=markable_81Min=377SemType=do)_))
devil ’s own game
(EntityID=85-PseudoMarkableID=markable_127Min=382SemType=quantifier_)(EntityID=86MarkableID=markable_80Min=382SemType=dn)
each coin I spend on my loved ones
(EntityID=87-PseudoMarkableID=markable_194Min=413,417SemType=quantifier(EntityID=10MarkableID=markable_129Min=391SemType=do)__)(EntityID=10MarkableID=markable_24Min=394SemType=do)
them
(EntityID=84MarkableID=markable_173Min=396SemType=doElementOf=100(EntityID=10MarkableID=markable_155Min=395SemType=do)_)
each I spend on myself
(EntityID=90-PseudoMarkableID=markable_63Min=420SemType=coordination(EntityID=54MarkableID=markable_197Min=440,441SemType=do)_(EntityID=91MarkableID=markable_30Min=420SemType=dn(EntityID=54MarkableID=markable_140Min=420SemType=do)))
I
(EntityID=94MarkableID=marka

(EntityID=35-PseudoMarkableID=markable_22Min=196SemType=coordination(EntityID=6MarkableID=markable_54Min=193SemType=do)_(EntityID=14MarkableID=markable_132Min=196SemType=do(EntityID=6MarkableID=markable_35Min=195SemType=do)))
turn
(EntityID=36MarkableID=markable_72Min=206SemType=dn_____(EntityID=31MarkableID=markable_113Min=205SemType=do)(EntityID=37MarkableID=markable_137Min=206SemType=dn_))
you and your community
(EntityID=38MarkableID=markable_90Min=214SemType=dn(EntityID=6MarkableID=markable_32Min=213SemType=do))
I
(EntityID=39MarkableID=markable_37Min=218SemType=dn(EntityID=29MarkableID=markable_28Min=217SemType=do)_)
my best
(EntityID=29MarkableID=markable_21Min=223SemType=do(EntityID=6MarkableID=markable_70Min=222SemType=do))
your kind words
(EntityID=40MarkableID=markable_55Min=232SemType=dn(EntityID=6MarkableID=markable_101Min=231SemType=do))
I
(EntityID=6MarkableID=markable_71Min=283SemType=do_(EntityID=6MarkableID=markable_66Min=281SemType=do)_)
you
(EntityID=12MarkableID=ma

(EntityID=70MarkableID=markable_107Min=423SemType=dn___(EntityID=71MarkableID=markable_63Min=423SemType=dn______(EntityID=72MarkableID=markable_105Min=427SemType=dn__)))(EntityID=73MarkableID=markable_32Min=427SemType=dn)
you
(EntityID=74MarkableID=markable_6Min=436SemType=dn(EntityID=26MarkableID=markable_69Min=435SemType=do))
that
(EntityID=76-PseudoMarkableID=markable_221Min=451,462SemType=coordination(EntityID=75MarkableID=markable_218Min=451,453SemType=dnElementOf=105(EntityID=77MarkableID=markable_232Min=452SemType=dn))(EntityID=78MarkableID=markable_219Min=454,458SemType=dnElementOf=105__(EntityID=80MarkableID=markable_104Min=449SemType=dn(EntityID=79MarkableID=markable_222Min=457SemType=dn)))_(EntityID=81MarkableID=markable_14Min=453SemType=dnElementOf=105(EntityID=82MarkableID=markable_220Min=461SemType=dn)))
You
(EntityID=83MarkableID=markable_182Min=462SemType=dn___(EntityID=74MarkableID=markable_168Min=460SemType=do(EntityID=26MarkableID=markable_26Min=459SemType=do))(Entit

(EntityID=51MarkableID=markable_156Min=338SemType=dn__)(EntityID=52MarkableID=markable_47Min=338SemType=dn)
Its
(EntityID=55MarkableID=markable_102Min=369SemType=dn___(EntityID=2MarkableID=markable_157Min=369SemType=do))
I
(EntityID=56-PseudoMarkableID=markable_16Min=377SemType=predicate_(EntityID=57MarkableID=markable_58Min=376SemType=do)__)
that
(EntityID=58MarkableID=markable_193Min=386,387SemType=dn(EntityID=6MarkableID=markable_96Min=384SemType=do))
a war we should wage together
(EntityID=59MarkableID=markable_195Min=399,410SemType=dn______(EntityID=60MarkableID=markable_136Min=405SemType=dn(EntityID=59MarkableID=markable_21Min=404SemType=do))_(EntityID=61MarkableID=markable_45Min=407SemType=dn))
us
(EntityID=62MarkableID=markable_161Min=412SemType=dn(EntityID=59MarkableID=markable_39Min=411SemType=do))
all those who vowed to only fulfill their oaths when pigs fly
(EntityID=63MarkableID=markable_164Min=421SemType=dn__(EntityID=2MarkableID=markable_137Min=421SemType=do)___(EntityID

In [251]:
index_total = []
for index, entity in enumerate(entities):
    entity_index = entity_df.index[(entity_df['entity']==entity)].tolist()[0]
    print("entity:", entity)

    idx_li = []
    identity = []
    for token in entity.split(" "):
        print(token)
#         print(token_df.index[(token_df['form']==token)].tolist()[0]) # entity에 해당하는 token의 인덱스
        
        # token 중에서 unused_token인 것은 제외 (entity로 사용되지 않은 token 제외)
#         index = token_df.index[(token_df['form']==token)].tolist()[0]
        index = token_df.index[(token_df['identity'].apply(lambda x:x != ['_'])) & (token_df['form']==token)].tolist()[0]

        idx_li.append(index)
        index_total.append(index)
        identity += token_df.loc[index,'identity']
    
    print('token index:', idx_li)
    
    identity_to_string = ''.join(identity)
    print('identity:',identity_to_string)
    print("="*50)

entity: a ceiling
a
ceiling
token index: [1, 2]
identity: (EntityID=1MarkableID=markable_1Min=2SemType=dn)
entity: a tree
a
tree
token index: [1, 4]
identity: (EntityID=1MarkableID=markable_1Min=2SemType=dn)
entity: a floor
a
floor
token index: [1, 6]
identity: (EntityID=1MarkableID=markable_1Min=2SemType=dn)
entity: a bag
a
bag
token index: [1, 8]
identity: (EntityID=1MarkableID=markable_1Min=2SemType=dn)
entity: a shield
a
shield
token index: [1, 10]
identity: (EntityID=1MarkableID=markable_1Min=2SemType=dn)
entity: a sword
a
sword
token index: [1, 12]
identity: (EntityID=1MarkableID=markable_1Min=2SemType=dn)
entity: an armor
an
armor
token index: [13, 14]
identity: (EntityID=7MarkableID=markable_7Min=14SemType=dn)
entity: a shoes
a
shoes
token index: [1, 16]
identity: (EntityID=1MarkableID=markable_1Min=2SemType=dn)
entity: a vest
a
vest
token index: [1, 18]
identity: (EntityID=1MarkableID=markable_1Min=2SemType=dn)
entity: member
member
token index: [19]
identity: (EntityID=10Mark

IndexError: list index out of range

In [247]:
index_total = []
for index, entity in enumerate(entities):
    entity_index = entity_df.index[(entity_df['entity']==entity)].tolist()[0]
    print("entity:", entity)

    idx_li = []
    identity = []
    for token in entity.split(" "):
        print(token)
#         print(token_df.index[(token_df['form']==token)].tolist()[0]) # entity에 해당하는 token의 인덱스
        
        # token 중에서 unused_token인 것은 제외 (entity로 사용되지 않은 token 제외)
#         index = token_df.index[(token_df['form']==token)].tolist()[0]
        index = token_df.index[(token_df['identity'].apply(lambda x:x != ['_'])) & (token_df['form']==token)].tolist()[0]

        idx_li.append(index)
        index_total.append(index)
        identity += token_df.loc[index,'identity']
    
    print('token index:', idx_li)
    
    identity_to_string = ''.join(identity)
    print('identity:',identity_to_string)
    
    if identity_to_string.count('(') == 1 and identity_to_string.count(')') == 1:  # 중복되지 않는 entity
        identity_head = identity_to_string
        print(identity_head)
        token_df = token_df.drop(index=idx_li, axis=0)
#         token_df = token_df[token_df.index > idx_li[-1]]  
        print('deleted')
    
    elif identity_to_string.count('(') != identity_to_string.count(')'):  # entity 안에 있는 entity
#         token_df = token_df[token_df.index > idx_li[0]-1]
        identity_head = identity_to_string[identity_to_string.rindex('('):identity_to_string.index(')')+1]
        print(identity_head)
        
    else:
        identity_head = identity_to_string[:identity_to_string.rindex('(')]  # 큰 entity
        print(identity_head)
#         token_df = token_df[token_df.index > idx_li[-1]]
        token_df = token_df.drop(index=idx_li, axis=0)    
        print("deleted")
        
        
    # head
    pattern = re.compile(r"Min=\d+[,]{0,1}\d*[,]{0,1}\d*")
    head= re.findall(pattern, identity_head)[0]
    head_id = head.replace("Min=","")

#     entity_idx = entity_df.index[entity_df['entity']==entity].tolist()[0]
    entity_df.loc[entity_index, 'head'] = head_id
    
    
    print("="*20)
# token_df

entity: a ceiling
a
ceiling
token index: [1, 2]
identity: (EntityID=1MarkableID=markable_1Min=2SemType=dn)
(EntityID=1MarkableID=markable_1Min=2SemType=dn)
deleted
entity: a tree
a
tree
token index: [3, 4]
identity: (EntityID=2MarkableID=markable_2Min=4SemType=dn)
(EntityID=2MarkableID=markable_2Min=4SemType=dn)
deleted
entity: a floor
a
floor
token index: [5, 6]
identity: (EntityID=3MarkableID=markable_3Min=6SemType=dn)
(EntityID=3MarkableID=markable_3Min=6SemType=dn)
deleted
entity: a bag
a
bag
token index: [7, 8]
identity: (EntityID=4MarkableID=markable_4Min=8SemType=dn)
(EntityID=4MarkableID=markable_4Min=8SemType=dn)
deleted
entity: a shield
a
shield
token index: [9, 10]
identity: (EntityID=5MarkableID=markable_5Min=10SemType=dn)
(EntityID=5MarkableID=markable_5Min=10SemType=dn)
deleted
entity: a sword
a
sword
token index: [11, 12]
identity: (EntityID=6MarkableID=markable_6Min=12SemType=dn)
(EntityID=6MarkableID=markable_6Min=12SemType=dn)
deleted
entity: an armor
an
armor
token i

IndexError: list index out of range

In [243]:
entity_df.head(50)

,entity,index,head
0,a ceiling,0,2
1,a tree,0,4
2,a floor,0,6
3,a bag,0,8
4,a shield,0,10
5,a sword,0,12
6,an armor,0,14
7,a shoes,0,16
8,a vest,0,18
9,member,0,19


In [219]:
identity_head = "(EntityID=12MarkableID=markable_108Min=66,67SemType=dn"
pattern = re.compile(r"Min=\d+[,]{0,1}\d*[,]{0,1}\d*")
head= re.findall(pattern, identity_head)[0]
head

'Min=66,67'

In [223]:
head_id = head.replace("Min=","")
head_id

'66,67'

In [231]:
entity = 'a ceiling'
entity_idx = entity_df.index[entity_df['entity']==entity].tolist()[0]
entity_df.loc[entity_idx, 'head'] = head_id

In [232]:
entity_df

,entity,index,head
0,a ceiling,0,"66,67"
1,a tree,0,0
2,a floor,0,0
3,a bag,0,0
4,a shield,0,0
...,...,...,...
1569,the sick,0,0
1570,the lepers,0,0
1571,these golden halls,0,0
1572,you words,0,0


In [387]:
# head

entity = {}

open_stack_index = []
stack_token = []

head = []
data = """"""
max_count = 0
with open(file, 'r', encoding='utf-8') as read_file:
    for line in read_file:
        if line != "\n":
            data += line
        elif line == "\n":
            split_func = lambda line, i: line[i].split("|")
            sentences = parse(data,
                              fields=['id', 'form', 'lemma', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps',
                                      'misc', 'identity', 'bridging', 'discourse_deixis', 'reference', 'nom_sem'],
                              field_parsers={"identity": split_func, "nom_sem": split_func})

            sentence = sentences[0]
            for index, token in enumerate(sentence):
                identities = token['identity']
                for identity in identities:
                    if "(" in identity:
                        stack_token.append(str(token))
                        open_stack_index.append(index)
                    elif ")" in identity:
                        count = str(identity).count(")")
                        end_index = index
                        for i in range(0, count):
                            if len(open_stack_index) == 0:
                                print("Do not match !")
                            else:
                                start_index = open_stack_index.pop()
                                for i in range(start_index, end_index+1):
#                                     print(sentence[i]['identity'])
                                    if len(sentence[i]['identity']) == 1:
                                        continue
                                    head.append(sentence[i]["identity"])
#                                 head = head.strip()
#                                 if head not in entity:
#                                     entity[head] = len(entity)
#             data = """"""

In [584]:
# head
for h in head:
    stringh = ''.join(h)
    print(h)
    print(stringh)
    
    print("(:", stringh.count('('))
    print("):", stringh.count(')'))
    print("-"*20)

['(EntityID=1', 'MarkableID=markable_1', 'Min=2', 'SemType=dn']
(EntityID=1MarkableID=markable_1Min=2SemType=dn
(: 1
): 0
--------------------
['(EntityID=2', 'MarkableID=markable_2', 'Min=4', 'SemType=dn']
(EntityID=2MarkableID=markable_2Min=4SemType=dn
(: 1
): 0
--------------------
['(EntityID=3', 'MarkableID=markable_3', 'Min=6', 'SemType=dn']
(EntityID=3MarkableID=markable_3Min=6SemType=dn
(: 1
): 0
--------------------
['(EntityID=4', 'MarkableID=markable_4', 'Min=8', 'SemType=dn']
(EntityID=4MarkableID=markable_4Min=8SemType=dn
(: 1
): 0
--------------------
['(EntityID=5', 'MarkableID=markable_5', 'Min=10', 'SemType=dn']
(EntityID=5MarkableID=markable_5Min=10SemType=dn
(: 1
): 0
--------------------
['(EntityID=6', 'MarkableID=markable_6', 'Min=12', 'SemType=dn']
(EntityID=6MarkableID=markable_6Min=12SemType=dn
(: 1
): 0
--------------------
['(EntityID=7', 'MarkableID=markable_7', 'Min=14', 'SemType=dn']
(EntityID=7MarkableID=markable_7Min=14SemType=dn
(: 1
): 0
--------------

['(EntityID=10', 'MarkableID=markable_156', 'Min=2', 'SemType=do)']
(EntityID=10MarkableID=markable_156Min=2SemType=do)
(: 1
): 1
--------------------
['(EntityID=12', 'MarkableID=markable_108', 'Min=6', 'SemType=dn(EntityID=10', 'MarkableID=markable_44', 'Min=6', 'SemType=do)']
(EntityID=12MarkableID=markable_108Min=6SemType=dn(EntityID=10MarkableID=markable_44Min=6SemType=do)
(: 2
): 1
--------------------
['(EntityID=12', 'MarkableID=markable_108', 'Min=6', 'SemType=dn(EntityID=10', 'MarkableID=markable_44', 'Min=6', 'SemType=do)']
(EntityID=12MarkableID=markable_108Min=6SemType=dn(EntityID=10MarkableID=markable_44Min=6SemType=do)
(: 2
): 1
--------------------
['(EntityID=13', 'MarkableID=markable_32', 'Min=10', 'SemType=dn']
(EntityID=13MarkableID=markable_32Min=10SemType=dn
(: 1
): 0
--------------------
['(EntityID=14-Pseudo', 'MarkableID=markable_107', 'Min=13', 'SemType=quantifier)']
(EntityID=14-PseudoMarkableID=markable_107Min=13SemType=quantifier)
(: 1
): 1
----------------

['(EntityID=15', 'MarkableID=markable_26', 'Min=20', 'SemType=dn']
(EntityID=15MarkableID=markable_26Min=20SemType=dn
(: 1
): 0
--------------------
['(EntityID=10', 'MarkableID=markable_142', 'Min=22', 'SemType=do)']
(EntityID=10MarkableID=markable_142Min=22SemType=do)
(: 1
): 1
--------------------
['(EntityID=16', 'MarkableID=markable_53', 'Min=30', 'SemType=dn']
(EntityID=16MarkableID=markable_53Min=30SemType=dn
(: 1
): 0
--------------------
['(EntityID=17', 'MarkableID=markable_40', 'Min=33', 'SemType=dn)']
(EntityID=17MarkableID=markable_40Min=33SemType=dn)
(: 1
): 1
--------------------
['(EntityID=11', 'MarkableID=markable_100', 'Min=35', 'SemType=do)']
(EntityID=11MarkableID=markable_100Min=35SemType=do)
(: 1
): 1
--------------------
['(EntityID=10', 'MarkableID=markable_93', 'Min=38', 'SemType=do)']
(EntityID=10MarkableID=markable_93Min=38SemType=do)
(: 1
): 1
--------------------
['(EntityID=10', 'MarkableID=markable_94', 'Min=43', 'SemType=do)']
(EntityID=10MarkableID=mar

--------------------
['(EntityID=13', 'MarkableID=markable_47', 'Min=50', 'SemType=do(EntityID=11', 'MarkableID=markable_136', 'Min=49', 'SemType=do', 'ElementOf=23)']
(EntityID=13MarkableID=markable_47Min=50SemType=do(EntityID=11MarkableID=markable_136Min=49SemType=doElementOf=23)
(: 2
): 1
--------------------
['(EntityID=13', 'MarkableID=markable_47', 'Min=50', 'SemType=do(EntityID=11', 'MarkableID=markable_136', 'Min=49', 'SemType=do', 'ElementOf=23)']
(EntityID=13MarkableID=markable_47Min=50SemType=do(EntityID=11MarkableID=markable_136Min=49SemType=doElementOf=23)
(: 2
): 1
--------------------
['(EntityID=18', 'MarkableID=markable_132', 'Min=54', 'SemType=dn']
(EntityID=18MarkableID=markable_132Min=54SemType=dn
(: 1
): 0
--------------------
['(EntityID=13', 'MarkableID=markable_90', 'Min=59', 'SemType=do']
(EntityID=13MarkableID=markable_90Min=59SemType=do
(: 1
): 0
--------------------
['(EntityID=19', 'MarkableID=markable_162', 'Min=76,79', 'SemType=dn']
(EntityID=19MarkableID

['(EntityID=10', 'MarkableID=markable_128', 'Min=16', 'SemType=do)']
(EntityID=10MarkableID=markable_128Min=16SemType=do)
(: 1
): 1
--------------------
['(EntityID=15', 'MarkableID=markable_26', 'Min=20', 'SemType=dn']
(EntityID=15MarkableID=markable_26Min=20SemType=dn
(: 1
): 0
--------------------
['(EntityID=10', 'MarkableID=markable_142', 'Min=22', 'SemType=do)']
(EntityID=10MarkableID=markable_142Min=22SemType=do)
(: 1
): 1
--------------------
['(EntityID=16', 'MarkableID=markable_53', 'Min=30', 'SemType=dn']
(EntityID=16MarkableID=markable_53Min=30SemType=dn
(: 1
): 0
--------------------
['(EntityID=17', 'MarkableID=markable_40', 'Min=33', 'SemType=dn)']
(EntityID=17MarkableID=markable_40Min=33SemType=dn)
(: 1
): 1
--------------------
['(EntityID=11', 'MarkableID=markable_100', 'Min=35', 'SemType=do)']
(EntityID=11MarkableID=markable_100Min=35SemType=do)
(: 1
): 1
--------------------
['(EntityID=10', 'MarkableID=markable_93', 'Min=38', 'SemType=do)']
(EntityID=10MarkableID=m

...

['(EntityID=64', 'MarkableID=markable_31', 'Min=459', 'SemType=do)']
(EntityID=64MarkableID=markable_31Min=459SemType=do)
(: 1
): 1
--------------------
['(EntityID=65-Pseudo', 'MarkableID=markable_64', 'Min=468', 'SemType=quantifier']
(EntityID=65-PseudoMarkableID=markable_64Min=468SemType=quantifier
(: 1
): 0
--------------------
['(EntityID=66', 'MarkableID=markable_24', 'Min=465', 'SemType=dn']
(EntityID=66MarkableID=markable_24Min=465SemType=dn
(: 1
): 0
--------------------
['(EntityID=67', 'MarkableID=markable_125', 'Min=468', 'SemType=dn']
(EntityID=67MarkableID=markable_125Min=468SemType=dn
(: 1
): 0
--------------------
['(EntityID=11', 'MarkableID=markable_50', 'Min=470', 'SemType=do)']
(EntityID=11MarkableID=markable_50Min=470SemType=do)
(: 1
): 1
--------------------
['(EntityID=50', 'MarkableID=markable_113', 'Min=473', 'SemType=do)']
(EntityID=50MarkableID=markable_113Min=473SemType=do)
(: 1
): 1
--------------------
['(EntityID=13', 'MarkableID=markable_122', 'Min=480',

(EntityID=41-PseudoMarkableID=markable_43Min=270SemType=quantifier(EntityID=37MarkableID=markable_133Min=269SemType=do)
(: 2
): 1
--------------------
['(EntityID=41-Pseudo', 'MarkableID=markable_43', 'Min=270', 'SemType=quantifier(EntityID=37', 'MarkableID=markable_133', 'Min=269', 'SemType=do)']
(EntityID=41-PseudoMarkableID=markable_43Min=270SemType=quantifier(EntityID=37MarkableID=markable_133Min=269SemType=do)
(: 2
): 1
--------------------
[')(EntityID=43', 'MarkableID=markable_34', 'Min=275', 'SemType=dn)']
)(EntityID=43MarkableID=markable_34Min=275SemType=dn)
(: 1
): 2
--------------------
['(EntityID=44-Pseudo', 'MarkableID=markable_160', 'Min=298,299', 'SemType=quantifier']
(EntityID=44-PseudoMarkableID=markable_160Min=298,299SemType=quantifier
(: 1
): 0
--------------------
['(EntityID=37', 'MarkableID=markable_62', 'Min=283', 'SemType=do)']
(EntityID=37MarkableID=markable_62Min=283SemType=do)
(: 1
): 1
--------------------
['(EntityID=45-Pseudo', 'MarkableID=markable_33', '

--------------------
['(EntityID=13', 'MarkableID=markable_75', 'Min=79', 'SemType=do']
(EntityID=13MarkableID=markable_75Min=79SemType=do
(: 1
): 0
--------------------
['(EntityID=18', 'MarkableID=markable_115', 'Min=83', 'SemType=do)']
(EntityID=18MarkableID=markable_115Min=83SemType=do)
(: 1
): 1
--------------------
['(EntityID=10', 'MarkableID=markable_12', 'Min=85', 'SemType=do)']
(EntityID=10MarkableID=markable_12Min=85SemType=do)
(: 1
): 1
--------------------
['(EntityID=10', 'MarkableID=markable_154', 'Min=88', 'SemType=do)']
(EntityID=10MarkableID=markable_154Min=88SemType=do)
(: 1
): 1
--------------------
['(EntityID=12', 'MarkableID=markable_124', 'Min=96', 'SemType=do(EntityID=10', 'MarkableID=markable_20', 'Min=95', 'SemType=do)']
(EntityID=12MarkableID=markable_124Min=96SemType=do(EntityID=10MarkableID=markable_20Min=95SemType=do)
(: 2
): 1
--------------------
['(EntityID=12', 'MarkableID=markable_124', 'Min=96', 'SemType=do(EntityID=10', 'MarkableID=markable_20', 'M

(EntityID=11MarkableID=markable_50Min=470SemType=do)
(: 1
): 1
--------------------
['(EntityID=50', 'MarkableID=markable_113', 'Min=473', 'SemType=do)']
(EntityID=50MarkableID=markable_113Min=473SemType=do)
(: 1
): 1
--------------------
['(EntityID=13', 'MarkableID=markable_122', 'Min=480', 'SemType=do']
(EntityID=13MarkableID=markable_122Min=480SemType=do
(: 1
): 0
--------------------
['(EntityID=68', 'MarkableID=markable_118', 'Min=475', 'SemType=dn']
(EntityID=68MarkableID=markable_118Min=475SemType=dn
(: 1
): 0
--------------------
['(EntityID=13', 'MarkableID=markable_122', 'Min=480', 'SemType=do']

(EntityID=25MarkableID=markable_45Min=146SemType=dn
(: 1
): 0
--------------------
['(EntityID=24', 'MarkableID=markable_14', 'Min=146', 'SemType=dn']
(EntityID=24MarkableID=markable_14Min=146SemType=dn
(: 1
): 0
--------------------
['(EntityID=25', 'MarkableID=markable_45', 'Min=146', 'SemType=dn']
(EntityID=25MarkableID=markable_45Min=146SemType=dn
(: 1
): 0
--------------------


(EntityID=50MarkableID=markable_88Min=350SemType=do)
(: 1
): 1
--------------------
['(EntityID=12', 'MarkableID=markable_91', 'Min=357', 'SemType=do(EntityID=50', 'MarkableID=markable_135', 'Min=356', 'SemType=do)']
(EntityID=12MarkableID=markable_91Min=357SemType=do(EntityID=50MarkableID=markable_135Min=356SemType=do)
(: 2
): 1
--------------------
['(EntityID=12', 'MarkableID=markable_91', 'Min=357', 'SemType=do(EntityID=50', 'MarkableID=markable_135', 'Min=356', 'SemType=do)']
(EntityID=12MarkableID=markable_91Min=357SemType=do(EntityID=50MarkableID=markable_135Min=356SemType=do)
(: 2
): 1
--------------------
['(EntityID=13', 'MarkableID=markable_73', 'Min=360', 'SemType=do']
(EntityID=13MarkableID=markable_73Min=360SemType=do
(: 1
): 0
--------------------
['(EntityID=52', 'MarkableID=markable_161', 'Min=383', 'SemType=dn)']
(EntityID=52MarkableID=markable_161Min=383SemType=dn)
(: 1
): 1
--------------------
[')(EntityID=53', 'MarkableID=markable_130', 'Min=365', 'SemType=dn)']
)

In [102]:
en = ['(EntityID=13','MarkableID=markable_47','Min=50','SemType=do(EntityID=11', 'MarkableID=markable_136','Min=49','SemType=do','ElementOf=23)']
stringen = ''.join(en)
print(stringen)
l = re.findall('\(([^)]+)',stringen)
print(l)
l2 = re.findall('\(([^)]+)',l[0])
print(l2)

(EntityID=13MarkableID=markable_47Min=50SemType=do(EntityID=11MarkableID=markable_136Min=49SemType=doElementOf=23)
['EntityID=13MarkableID=markable_47Min=50SemType=do(EntityID=11MarkableID=markable_136Min=49SemType=doElementOf=23']
['EntityID=11MarkableID=markable_136Min=49SemType=doElementOf=23']


In [390]:
entity_df['head'] = ['0']*len(entity_df)
entity_df

,entity,index,head
0,a ceiling,"[1, 2]",0
1,a tree,0,0
2,a floor,0,0
3,a bag,0,0
4,a shield,0,0
...,...,...,...
1569,the sick,0,0
1570,the lepers,0,0
1571,these golden halls,0,0
1572,you words,0,0


In [392]:
len(head)

1864081

In [359]:
print(data)

# sent_id = episode_5953-47
# text = Buy them honey .
596   Buy    _  _  _  _  _  _  _  _  _                                                         _  _  _  
597   them   _  _  _  _  _  _  _  _  (EntityID=35|MarkableID=markable_81|Min=582|SemType=do)   _  _  _  (MarkableID=markable_81|Entity_Type=person|Genericity=generic-yes)
598   honey  _  _  _  _  _  _  _  _  (EntityID=95|MarkableID=markable_213|Min=598|SemType=dn)  _  _  _  (MarkableID=markable_213|Entity_Type=substance|Genericity=undersp-substance)
599   .      _  _  _  _  _  _  _  _  _                                                         _  _  _  



In [ ]:
# head

token = 'a'
index = df.index[(df['form']==token)].tolist()[0]
column_identity = df.loc[index]['identity']
print(column_identity)
# print(len(column_identity))
head_id = column_identity[2].replace("Min=","")
# head_id

df.loc[index, 'head'] = head_id
# df = df.drop(index = index)
df

In [ ]:
for idx in range(1,token_df.shape[0]+1):
    column_identity = token_df.loc[idx]['identity']
    print(column_identity)
#     if len(column_identity) < 3:
#         continue
#     head_id = column_identity[2].replace("Min=","")
#     token_df.loc[index, 'head'] = head_id
# token_df

In [ ]:
for index, token in enumerate(sentence):
    
    index = df.index[(df['form']==token)].tolist()[0]
    column_identity = df.loc([index])['identity']
    
    if len(column_identity) != 1:
        head_id = column_identity[2].replace('Min=','')
        df.loc[index, 'head'] = head_id